Example on how to run the python203 package 

In [13]:
import logging
import pandas as pd
from datetime import datetime
from python203.volumebacktest import VolumeBacktest, Tradesignals
from python203.backtestevaluator import BacktestEvaluator, BacktestRating
from python203.Tokentransfers import TokenTransfer, contract
from pybacktestchain.data_module import get_stocks_data
from web3 import Web3

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Example Integration Script
def main():
    # Step 1: Initialize VolumeBacktest
    backtest = VolumeBacktest(
        initial_date=datetime(2019, 1, 1),
        final_date=datetime(2022, 1, 1),
        universe=['AAPL', 'MSFT', 'GOOGL'],
        initial_cash=1_000_000,
        verbose=True
    )

    # Fetch market data
    data = get_stocks_data(
        backtest.universe,
        backtest.initial_date.strftime('%Y-%m-%d'),
        backtest.final_date.strftime('%Y-%m-%d')
    )

    if data.empty:
        logging.error("No market data available.")
        return

    # Generate signals and execute trades
    data_with_signals = backtest.compute_signals(data)
    trades = Tradesignals(broker=backtest.broker).execute_trades(data_with_signals)

    # Step 2: Evaluate Backtest
    evaluator = BacktestEvaluator()
    metrics = evaluator.compute_metrics(trades)

    # Generate rating
    rater = BacktestRating()
    rating = rater.rate_backtest(metrics)
    logging.info(f"Backtest Metrics: {metrics}")
    logging.info(f"Backtest Rating: {rating}")

    # Step 3: Transfer Tokens Based on Rating
    ISSUER_PRIVATE_KEY = "3a078e0c831b014a8ff42a99b0e58718eecd856abf4ca040eaada338b9c33077"
    ISSUER_ADDRESS = Web3.to_checksum_address("0x0390cF896B4a7D984017e6C9d3d17b5A6287a874")
    USER_ADDRESS = Web3.to_checksum_address("0xc2142C6007227C9f1F1C6fe6cb7181Ce491F37a7")

    token_transfer = TokenTransfer(private_key=ISSUER_PRIVATE_KEY, sender_address=ISSUER_ADDRESS)

    try:
        token_transfer.transfer_tokens_from_rating(USER_ADDRESS, rating)
        logging.info("Token transfer completed successfully.")
    except Exception as e:
        logging.error(f"Token transfer failed: {e}")
        return

    # Step 4: Log Token Balances and Total Supply
    try:
        # Fetch balances and total supply
        issuer_balance = contract.functions.balanceOf(ISSUER_ADDRESS).call()
        user_balance = contract.functions.balanceOf(USER_ADDRESS).call()
        total_supply = contract.functions.totalSupply().call()

        logging.info(f"Issuer Token Balance: {Web3.from_wei(issuer_balance, 'ether')} tokens")
        logging.info(f"User Token Balance: {Web3.from_wei(user_balance, 'ether')} tokens")
        logging.info(f"Total Token Supply: {Web3.from_wei(total_supply, 'ether')} tokens")
    except Exception as e:
        logging.error(f"Error fetching token balances or total supply: {e}")

# Run the main function
if __name__ == "__main__":
    main()


INFO:root:Generating volume-based signals...
c:\Users\ASUS\anaconda3\Lib\site-packages\pybacktestchain\broker.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.transaction_log = pd.concat([self.transaction_log, transaction], ignore_index=True)
INFO:root:Detailed signals for the first 5 days:
INFO:root:{'Date': Timestamp('2019-01-30 00:00:00-0500', tz='America/New_York'), 'Ticker': 'AAPL', 'Signal': 'BUY', 'Price': 39.4588737487793}
INFO:root:{'Date': Timestamp('2019-01-30 00:00:00-0500', tz='America/New_York'), 'Ticker': 'MSFT', 'Signal': 'BUY', 'Price': 100.2683334350586}
INFO:root:{'Date': Timestamp('2019-01-31 00:00:00-0500', tz='America/New_York'), 'Ticker': 'MSFT', 'Signal': 'BUY', 'Price': 98.43036651611328}
INFO:root:{'Date'